# Алгоритм Витерби

## Решётка

Решёткой (англ. trellis) называется граф, обладающий следующими свойствами:
* Вершины разбиты на $n$ непересекающихся подмножеств (уровни или ярусы $\{L_i\}$).
* $L_0 = \{v_{start}\}$ и $L_n = \{v_{terminal}\}$ — начальный и терминальный уровни.
* Граф направленный, допускаются рёбра между вершинами из $L_i$ в $L_j$ только если $i < j$.
* Рёбрам графа приписаны метки $f \in \mathbb{F}_q$, соответствующие символам кодовых слов.
* При прохождении по некоторому пути для принятого вектора $y$ на каждом ребре $e$ между ярусами $L_{i+1}$ и $L_i$ высчитывается длина ребра $d(f_e, y_i)$.

Такая конструкция задаёт некоторое множество кодовых слов $\mathcal{C}$ в пространстве $\mathbb{F}_q^n$, что соответствует линейным блоковым кодам. Если же устремить число ярусов в бесконечность, то такая решётка уже будет соответствовать свёрточным кодам. На практике для свёрточных кодов полезно разбивать код на сегменты (например, константной длины), и после этого возвращать положение в решётке к самой верхней вершине в ярусе.

Ниже приведён пример решётки (из интернета):

![Trellis example](trellis-example.png)

### Минимальная решётка

Профиль сложности решётки это последовательность ${\xi_i}$, где $\xi_i = |L_i|$. Минимальной решёткой $\mathcal{C}$ называется такая решётка $\mathcal{C}$, что её профиль минимальный (по метрике Евклида).

## Декодирование по решётке

Задача декодирования кодовых слов из $\mathcal{C}$ по критерию минимального расстояния сводится к поиску кратчайшего пути между терминальными вершинами. Алгоритм следующий:

1. Для каждого ребра $e$ вычислить его длину $d_e = d(f_e, y_i)$.
2. Для каждой вершины вычислить длину пути, оканчивающейся этой вершиной:
    * База: $d_{v_{start}} = 0$
    * Индукция: $d_v = \min_{e = (u, v)}(d_u + d_e)$, для каждой вершины так же запомним $u_v^{best} = \underset{u}{\operatorname{argmin}}(d_u + d_{(u, v)})$
3. Далее найдём кратчайший путь:
    * База: $w_n = v_{terminal}$
    * Индукция: $w_{i - 1} = u_{w_i}^{best}$
4. Результатом декодирования будет $\hat{c}$, что $\hat{c_i} = f_{(w_{i - 1}, w_i)}$.

## Построение минимальной решётки по порождающей матрице линейного блокового кода

Алгоритм построения минимальной решётки по матрице $G$:

1. Приводим матрицу $G$ к минимальной спэновой форме (далее считаем, что $G$ находится в ней).
2. Для каждой $i$-й строки матрицы $G$ находим первый активный (ненулевой) символ на позиции $s_i$ и последний активный символ на позиции $t_i$.
3. Для каждого яруса $L_j$ составляем список активных строк $X_j = \{i | s_i \le j < t_i\}$.
4. Из предыдущего шага следует, что каждому ярусу $L_j$ соответствует $|X_j| = \xi_j$ строк. Тогда заполним все ярусы всеми возможными вершинами из $\mathbb{F}_q^{\xi_j}$.
5. Введём ещё одну индексацию: пусть $u \in L_j$, тогда обозначим $u^{(X_j[i])} = u_i$. Пример: если $X_j = [1, 3, 7]$, то $u^{(1)} = u_0$, $u^{(3)} = u_1$, $u^{(7)} = u_2$.
6. Соединяем вершины $u \in L_{j-1}$ и $v \in L_j$, если для всех $i \in X_{j-1} \cap X_{j}$ выполняется $u^{(i)} = v^{(i)}$.
7. Для каждого ребра между вершинами $u \in L_{j-1}$ и $v \in L_j$ в качестве метки возьмём скалярное произведение $f_{(u, v)} = w \cdot x$ в поле $\mathbb{F}$, где:
    - Если $i \in X_{j-1}$, то $w^{(i)} = u^{(i)}$, а если $i \in X_j$, то $w^{(i)} = v^{(i)}$
    - $x_i = G[i][j]$, если $i \in X_{j-1} \cup X_{j}$

In [1]:
from algebraic.linear.combining import reed_muller
from tree.viterbi import get_active_rows, get_xi, trellis_from_gen
from algebraic.matrix import minimal_span_form
from algebraic.binary import Binary

g = reed_muller(1, 3)
print("g:")
print(g)

msg = minimal_span_form(g)
print("g in minimal span form:")
print(msg)

g:
[[1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]]
g in minimal span form:
[[1 1 1 1 0 0 0 0]
 [0 1 0 1 1 0 1 0]
 [0 0 1 1 1 1 0 0]
 [0 0 0 0 1 1 1 1]]


In [2]:
active_rows = get_active_rows(msg)
print("active rows:", active_rows)

xi = get_xi(active_rows)
print("xi:", xi)

active rows: [[], [0], [0, 1], [0, 1, 2], [1, 2], [1, 2, 3], [1, 3], [3], []]
xi: [0, 1, 2, 3, 2, 3, 2, 1, 0]


In [3]:
print("trellis in form (u, v, f):")
trellis = trellis_from_gen(g, one=Binary(1))
for layer in trellis:
    print(layer)

trellis in form (u, v, f):
[(*, 0, 0), (*, 1, 1)]
[(0, 00, 0), (0, 01, 1), (1, 10, 1), (1, 11, 0)]
[(00, 000, 0), (00, 001, 1), (10, 100, 1), (10, 101, 0), (01, 010, 0), (01, 011, 1), (11, 110, 1), (11, 111, 0)]
[(000, 00, 0), (100, 00, 1), (010, 10, 1), (110, 10, 0), (001, 01, 1), (101, 01, 0), (011, 11, 0), (111, 11, 1)]
[(00, 000, 0), (00, 001, 1), (10, 100, 1), (10, 101, 0), (01, 010, 1), (01, 011, 0), (11, 110, 0), (11, 111, 1)]
[(000, 00, 0), (100, 10, 0), (010, 00, 1), (110, 10, 1), (001, 01, 1), (101, 11, 1), (011, 01, 0), (111, 11, 0)]
[(00, 0, 0), (10, 0, 1), (01, 1, 1), (11, 1, 0)]
[(0, *, 0), (1, *, 1)]


Визуализация решётки:

![gen-lattice](gen-lattice.png)

## Построение минимальной решётки по проверочной матрице линейного блокового кода

Алгоритм построения минимальной решётки по матрице $H$:

1. Приводим матрицу $G$ к минимальной спэновой форме (необязательный шаг).
2. Каждый ярус заполняем всеми возможными вершинами из $\mathbb{F}_q^k$.
3. Начиная с вершины $\{0 0 \dots 0\}$ из яруса $L_0$ и далее для каждой вершины $u \in L_{j - 1}$, в которую мы уже проводили ребро, проводим $q$ рёбер в вершины $v(i) = u + i \cdot G[:][j]$ из яруса $L_j$, где $G[:][j]$ — $j$-ый столбец матрицы $G$.
4. В качестве метки ребра выбираем $f_{(u, v(i))} = i$.
5. В конце удаляем все рёбра, любой путь по которым не ведёт к вершине $\{0 0 \dots 0\}$ из яруса $L_n$.
